In [1]:
# Requires Pip Install
# pip install xgboost

In [2]:
# Import the data
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE

og_df = pd.read_csv('./original dataset/fake_job_postings.csv')
og_form = pd.read_csv('./formatted dataset/fake_job_postings.csv')

df = og_df.copy()
form = og_form.copy()

display(df.head())

display(form.head())

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


,job_id,title,country,state,city,department,salary_min,salary_max,company_profile,description,...,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,us,ny,new york,Marketing,-1.0,-1.0,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",...,NaN,0,1,0,Other,Internship,Undefined,Undefined,Marketing,0
1,2,Customer Service - Cloud Video Production,nz,NaN,auckland,Success,-1.0,-1.0,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,Undefined,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),us,ia,wever,Undefined,-1.0,-1.0,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",...,NaN,0,1,0,Undefined,Undefined,Undefined,Undefined,Undefined,0
3,4,Account Executive - Washington DC,us,dc,washington,Sales,-1.0,-1.0,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,...,Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,us,fl,fort worth,Undefined,-1.0,-1.0,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [3]:
# # moving below, this was too early and causing datal leakage

# # implement one hot encoding for department
# ohe = OneHotEncoder(min_frequency=20, handle_unknown="ignore")
# # One-hot encode the 'department' column
# ohe = OneHotEncoder(min_frequency=20, handle_unknown="ignore")
# dept_encoded = ohe.fit_transform(df[['department']])
# # Convert the sparse matrix to a dense array
# dept_encoded_dense = dept_encoded.toarray()
# # Create a DataFrame with the encoded department data
# dept_df = pd.DataFrame(dept_encoded_dense, columns=ohe.get_feature_names_out(["department"]))
# # Join the original DataFrame with the encoded department DataFrame
# df = df.join(dept_df)
# df.drop(['department'], axis=1, inplace=True)

In [4]:
data_leak = ['job_id']
form.drop(data_leak, axis=1, inplace=True)
df.drop(data_leak, axis=1, inplace=True)

# removing the categorical columns for a baseline test
df.drop(['company_profile','description','requirements','benefits','location','salary_range','title'], axis=1, inplace=True)
form.drop(['company_profile','description','requirements','benefits','title'], axis=1, inplace=True)

# creating a df with some categorical fields that aren't long text fields
categorical=['employment_type', 'required_experience', 'required_education', 'industry', 'function']
df_categorical = df.copy()
# form_cat = form.copy()

# Drop the remaining categorical columns from the original DataFrame
df.drop(categorical, axis=1, inplace=True)
form.drop(categorical, axis=1, inplace=True)

df_categorical = df_categorical.fillna('Unknown')
form['country'] = form['country'].fillna('Unknown')
df['department'] = df['department'].fillna('Unknown')
df = df.fillna(0)

# set the type to category for the XGB DMatrix to handle when flagging for categorical columns
df_categorical[categorical] = df_categorical[categorical].astype('category')
# form_cat[categorical] = form_cat[categorical].astype('category')
df_categorical[categorical] = df_categorical[categorical].astype('category')

# Train Test Split

In [5]:
# most basic model

x = df.drop(['fraudulent'], axis=1)
y = df['fraudulent']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

# build with oversampling

x_os = df.drop(['fraudulent'], axis=1)
y_os = df['fraudulent']

x_train_os, x_test_os, y_train_os, y_test_os = train_test_split(x_os, y_os, test_size=0.2, random_state=0)

# # Apply SMOTE to the training data
# smote = SMOTE(random_state=42)
# x_train_os, y_train_os = smote.fit_resample(x_train_os, y_train_os)

# build is oversampling and categorical fields

x_os_cat = df_categorical.drop(['fraudulent'], axis=1)
y_os_cat = df_categorical['fraudulent']

x_train_os_cat, x_test_os_cat, y_train_os_cat, y_test_os_cat = train_test_split(x_os_cat, y_os_cat, test_size=0.2, random_state=0)

# build with oversampling with formatted data

x_form = form.drop(['fraudulent'], axis=1)
y_form = form['fraudulent']

x_train_form, x_test_form, y_train_form, y_test_form = train_test_split(x_form, y_form, test_size=0.2, random_state=0)

# Transform data

In [6]:
# implement one hot encoding for department
ohe = OneHotEncoder(min_frequency=20, handle_unknown="ignore")

dept_encoded = ohe.fit_transform(x_train[['department']])
dept_encoded_dense = dept_encoded.toarray()
dept_df = pd.DataFrame(dept_encoded_dense, columns=ohe.get_feature_names_out(["department"]))
x_train = x_train.join(dept_df.fillna(-1))
x_train.drop(['department'], axis=1, inplace=True)

dept_encoded = ohe.transform(x_test[['department']])
dept_encoded_dense = dept_encoded.toarray()
dept_df = pd.DataFrame(dept_encoded_dense, columns=ohe.get_feature_names_out(["department"]))
x_test = x_test.join(dept_df.fillna(-1))
x_test.drop(['department'], axis=1, inplace=True)

dept_encoded = ohe.fit_transform(x_train_os[['department']])
dept_encoded_dense = dept_encoded.toarray()
dept_df = pd.DataFrame(dept_encoded_dense, columns=ohe.get_feature_names_out(["department"]))
x_train_os = x_train_os.join(dept_df.fillna(-1))
x_train_os.drop(['department'], axis=1, inplace=True)

dept_encoded = ohe.transform(x_test_os[['department']])
dept_encoded_dense = dept_encoded.toarray()
dept_df = pd.DataFrame(dept_encoded_dense, columns=ohe.get_feature_names_out(["department"]))
x_test_os = x_test_os.join(dept_df.fillna(-1))
x_test_os.drop(['department'], axis=1, inplace=True)

dept_encoded = ohe.fit_transform(x_train_os_cat[['department']])
dept_encoded_dense = dept_encoded.toarray()
dept_df = pd.DataFrame(dept_encoded_dense, columns=ohe.get_feature_names_out(["department"]))
dept_df = dept_df.fillna(-1)
x_train_os_cat = x_train_os_cat.join(dept_df)
x_train_os_cat.drop(['department'], axis=1, inplace=True)
# x_train_os_cat = x_train_os_cat.fillna(-1)

dept_encoded = ohe.transform(x_test_os_cat[['department']])
dept_encoded_dense = dept_encoded.toarray()
dept_df = pd.DataFrame(dept_encoded_dense, columns=ohe.get_feature_names_out(["department"]))
x_test_os_cat = x_test_os_cat.join(dept_df.fillna(-1))
x_test_os_cat.drop(['department'], axis=1, inplace=True)

dept_encoded = ohe.fit_transform(x_train_form[['department']])
dept_encoded_dense = dept_encoded.toarray()
dept_df = pd.DataFrame(dept_encoded_dense, columns=ohe.get_feature_names_out(["department"]))
x_train_form = x_train_form.join(dept_df.fillna(-1))
x_train_form.drop(['department'], axis=1, inplace=True)

dept_encoded = ohe.transform(x_test_form[['department']])
dept_encoded_dense = dept_encoded.toarray()
dept_df = pd.DataFrame(dept_encoded_dense, columns=ohe.get_feature_names_out(["department"]))
x_test_form = x_test_form.join(dept_df.fillna(-1))
x_test_form.drop(['department'], axis=1, inplace=True)

x_train = x_train.fillna(-1)
x_test = x_test.fillna(-1)
x_train_os = x_train_os.fillna(-1)
x_test_os = x_test_os.fillna(-1)
x_train_form = x_train_form.fillna(-1)
x_test_form = x_test_form.fillna(-1)

x_train.head()

,telecommuting,has_company_logo,has_questions,department_Accounting,department_Administration,department_Administrative,department_Business Development,department_CSD Relay,department_Clerical,department_Client Services,...,department_Production,department_R&D,department_Retail,department_Sales,department_Tech,department_Technical,department_Technology,department_Unknown,department_tech,department_infrequent_sklearn
14360,0,1,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
7053,0,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
17870,0,1,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
9228,0,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4160,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [7]:
# Extra cleanup for the formatted dataset

x_train_form[['city', 'state', 'country']] = x_train_form[['city', 'state', 'country']].fillna("Unknown").astype('category')
x_test_form[['city', 'state', 'country']] = x_test_form[['city', 'state', 'country']].fillna("Unknown").astype('category')

x_test_form.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3576 entries, 9256 to 11512
Data columns (total 42 columns):
 #   Column                             Non-Null Count  Dtype   
---  ------                             --------------  -----   
 0   country                            3576 non-null   category
 1   state                              3576 non-null   category
 2   city                               3576 non-null   category
 3   salary_min                         3576 non-null   float64 
 4   salary_max                         3576 non-null   float64 
 5   telecommuting                      3576 non-null   int64   
 6   has_company_logo                   3576 non-null   int64   
 7   has_questions                      3576 non-null   int64   
 8   department_Accounting              3576 non-null   float64 
 9   department_Administration          3576 non-null   float64 
 10  department_Administrative          3576 non-null   float64 
 11  department_Business Development    3576 non-

In [8]:
# Apply SMOTE to the training data now that the department column has been one-hot encoded
smote = SMOTE(random_state=42)

# x_train_os = x_train_os.replace([None, ''], np.nan)
x_train_resampled, y_train_resampled = smote.fit_resample(x_train_os, y_train_os)

In [9]:
# Encode categorical variables in x_train_form
# x_train_form_encoded = pd.get_dummies(x_train_form)
x_train_os_cat_encoded = pd.get_dummies(x_train_os_cat)

# also update x_test for same variables
# x_test_form_encoded = pd.get_dummies(x_test_form)
x_test_os_cat_encoded = pd.get_dummies(x_test_os_cat)

# getting errors that x_train_os_cat_encoded has NaN's
x_train_os_cat_encoded.fillna(-1, inplace=True)

# Apply SMOTE to the encoded training data
x_train_os_cat_resampled, y_train_os_cat_resampled = smote.fit_resample(x_train_os_cat_encoded, y_train_os_cat)

# Apply SMOTE to the encoded training data
# x_train_form_resampled, y_train_form_resampled = smote.fit_resample(x_train_form_encoded, y_train_form)


# Fit Models with train and test data

In [11]:
# Use test and train dataframes to create a DMatrix

dtrain = xgb.DMatrix(x_train, label=y_train)
dtest = xgb.DMatrix(x_test, label=y_test)

dtrain_os = xgb.DMatrix(x_train_resampled, label=y_train_resampled)
dtest_os = xgb.DMatrix(x_test_os, label=y_test_os)

dtrain_os_cat = xgb.DMatrix(x_train_os_cat_resampled, label=y_train_os_cat_resampled, enable_categorical=True)
dtest_os_cat = xgb.DMatrix(x_test_os_cat_encoded, label=y_test_os_cat, enable_categorical=True)

dtrain_form = xgb.DMatrix(x_train_form, label=y_train_form, enable_categorical=True)
dtest_form = xgb.DMatrix(x_test_form, label=y_test_form, enable_categorical=True)


# Train Models

In [12]:
# set params and train the DMatrix
params = {'max_depth': 2, 'eta': 1, 'objective': 'binary:logistic'}
num_round = 10

xg_model = xgb.train(params, dtrain, num_round)
xg_model_os = xgb.train(params, dtrain_os, num_round)
xg_model_os_cat = xgb.train(params, dtrain_os_cat, num_round)
xg_model_form = xgb.train(params, dtrain_form, num_round)

# Predict

In [13]:
# predict the xg model
preds = xg_model.predict(dtest)
preds_os = xg_model_os.predict(dtest_os)
preds_os_cat = xg_model_os_cat.predict(dtest_os_cat)
preds_form = xg_model_form.predict(dtest_form)

# convert the predictions to binary
preds_binary = (preds >= 0.5).astype(int)
preds_binary_os = (preds_os >= 0.5).astype(int)
preds_binary_os_cat = (preds_os_cat >= 0.5).astype(int)
preds_binary_form = (preds_form >= 0.5).astype(int)

# check accuracy of the model
print('Base Model: ', balanced_accuracy_score(y_test, preds_binary))
print('Oversampled Model: ', balanced_accuracy_score(y_test_os, preds_binary_os))
print('Oversample Categorized Model: ', balanced_accuracy_score(y_test_os_cat, preds_binary_os_cat))
print('Formatted Model: ', balanced_accuracy_score(y_test_form, preds_binary_form))

Base Model:  0.508489285284666
Oversampled Model:  0.7249145056795725
Oversample Categorized Model:  0.804205117629874


# Review Predictions

In [ ]:
# Calculate the confusion matrix
cm = confusion_matrix(y_test, preds_binary)
print('Confusion Matrix:')
print(cm)

# Display the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1])
disp.plot(cmap=plt.cm.Blues)
plt.show()

In [ ]:
# Calculate the confusion matrix
cm = confusion_matrix(y_test_os, preds_binary_os)
print('Confusion Matrix:')
print(cm)

# Display the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1])
disp.plot(cmap=plt.cm.Blues)
plt.show()

In [ ]:
# Calculate the confusion matrix
cm = confusion_matrix(y_test_os_cat, preds_binary_os_cat)
print('Confusion Matrix:')
print(cm)

# Display the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1])
disp.plot(cmap=plt.cm.Blues)
plt.show()

# Introduce XGBClassifier model

In [ ]:
from xgboost import XGBClassifier, XGBRegressor

xgbc = XGBClassifier(n_estimators=136, max_depth=4, learning_rate=.13, random_state=1)

# fit the model
xgbc.fit(x_test_os, y_test_os)

# Make predictions
train_pred_xgbc = xgbc.predict(x_train_os)
test_pred_xgbc = xgbc.predict(x_test_os)

train_pred_xgbc_form = xgbc.predict(x_train_form)
test_pred_xgbc_form = xgbc.predict(x_train_form)

In [ ]:

print('\nRandom Oversampled with Tuned XGB Classifer')
# Print scores
print("--------------------------------------------------------")
print(balanced_accuracy_score(y_train_os, train_pred_xgbc),'training score')
print(balanced_accuracy_score(y_test_os, test_pred_xgbc),'testing score')

print('\nRandom Oversampled and formatted Data with Tuned XGB Classifer')
# Print scores
print("--------------------------------------------------------")
print(balanced_accuracy_score(y_train_os, train_pred_xgbc),'training score')
print(balanced_accuracy_score(y_test_os, test_pred_xgbc),'testing score')